# Running the repo

You might have to add a HF token. Please don't steal my token!

If you want to edit any of the .py files, like benchmark.py or prompts.py, you can edit them directly in colab (Files [on the left bar] -> My Drive -> ....), but then you have to restart the runtime. (runtime -> restart session). Note that you don't have to do the pip install cell afterwards, since the runtime stil exists, it just got reset. Only if you disconnect/delete the runtime (such as if you turn your computer off and come back a few hours later) will you have to re-run the pip install cell.

In [1]:
from dotenv import load_dotenv
from huggingface_hub import login
import os

load_dotenv()
HF_TOKEN = os.environ.get("HF_TOKEN")
# Paste your token inside the quotes
login(HF_TOKEN)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
import importlib
import benchmark
import llm_agent
from llm_providers import create_llm
importlib.reload(benchmark)
importlib.reload(llm_agent)

<module 'llm_agent' from '/Users/jooseunglee/Desktop/codenames_llm/llm_agent.py'>

## Model Configuration

This cell sets up the LLM model for both codemaster and guesser agents.

**To switch models:** Simply change `MODEL_CHOICE` to one of:
- `"llama31"` - LLaMA 3.1 8B Instruct (default)
- `"qwen3"` - Qwen 2.5 7B Instruct
- `"mistral"` - Mistral 7B Instruct v0.2

**Key improvements:**
- Uses proper chat templates for each model (automatically detected)
- Supports 4-bit quantization to save memory
- Shared LLM instance for both agents (memory efficient)

In [3]:
import llm_agent
from llm_providers import create_llm

# ===== MODEL CONFIGURATION =====
# Choose which model to use by setting MODEL_CHOICE
# Options: "llama31", "qwen3", "mistral"

MODEL_CHOICE = "llama31"  # Change this to switch models

MODEL_CONFIGS = {
    "llama31": {
        "type": "local_hf",
        "model_name": "meta-llama/Llama-3.1-8B-Instruct",
        "temperature": 0.7,
        "max_tokens": 512,
        # "load_in_4bit": True
    },
    "qwen3": {
        "type": "local_hf",
        "model_name": "Qwen/Qwen3-8B",
        "temperature": 0.7,
        "max_tokens": 2048,  # 10x increase from default 200
        "disable_reasoning": True,  # Suppress <think> tags
    },
    "mistral": {
        "type": "local_hf",
        "model_name": "mistralai/Mistral-7B-Instruct-v0.2",
        "temperature": 0.7,
        # "load_in_4bit": True
    }
}

# Get the selected config
config = MODEL_CONFIGS[MODEL_CHOICE]
print(f"Loading model: {config['model_name']}")

# Create shared LLM instance
shared_llm_instance = create_llm(config)

# Create agents with shared instance
codemaster = llm_agent.LLMAgent(model_config=config, llm_instance=shared_llm_instance)
codemaster.initialize_role('codemaster')

guesser = llm_agent.LLMAgent(model_config=config, llm_instance=shared_llm_instance)
guesser.initialize_role('guesser')

print(f"✓ Codemaster and Guesser initialized with {MODEL_CHOICE}")

Loading model: meta-llama/Llama-3.1-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✓ Codemaster and Guesser initialized with llama31


In [4]:
bnch = benchmark.CodeNamesBenchmark()

# Run collaborative game with current model
print(f"\n{'='*60}")
print(f"Running game with {MODEL_CHOICE}: {config['model_name']}")
print(f"{'='*60}\n")

results = bnch.run_collab_matchup(config, num_games=1, codemaster=codemaster, guesser=guesser)

print(f"\n{'='*60}")
print(f"Results for {MODEL_CHOICE}")
print(f"{'='*60}")
print(results)

# Optionally save logs
# import os
# os.makedirs("logs", exist_ok=True)
# benchmark.export_turn_logs(f"logs/codenames_turns_{MODEL_CHOICE}.json")
# print(f"\nTurn logs saved to logs/codenames_turns_{MODEL_CHOICE}.json")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Running game with llama31: meta-llama/Llama-3.1-8B-Instruct


Starting collaborative game 0

Board State:
MOSCOW | NEW YORK | SOLDIER | RAY | STREAM
ENGINE | LINK | POLICE | BOMB | NOTE
SLIP | GOLD | SCALE | CAPITAL | POOL
DATE | FILE | WHALE | NOVEL | WAVE
STAFF | LIFE | CZECH | CRASH | SUIT


=== Turn 1 ===
Remaining words to guess: DATE, GOLD, MOSCOW, NOTE, SUIT, NEW YORK, SOLDIER, ENGINE, POOL


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


codemaster raw output: ```json
{
  "reasoning": "Using a common association to connect words related to a physical object",
  "clue": "TOOL",
  "number": 3
}
Codemaster's clue: TOOL 3
(Reasoning: Using a common association to connect words related to a physical object)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

guesser raw output: ```json
{
  "reasoning": "The clue is 'HOLIDAY 1' which means there is only one word related to 'HOLIDAY'. Given that we have already found words like 'DATE' which is related to calendar, I'm going to take a chance on 'CAPITAL' which is a related concept to holiday, often referring to a country's capital.",
  "guess": "CAPITAL"
}
Guesser's guess: CAPITAL
(Reasoning: The clue is 'HOLIDAY 1' which means there is only one word related to 'HOLIDAY'. Given that we have already found words like 'DATE' which is related to calendar, I'm going to take a chance on 'CAPITAL' which is a related concept to holiday, often referring to a country's capital.)
Hit a neutral word.

Board State:
[MOSCOW] | NEW YORK | [SOLDIER] | [RAY] | [STREAM]
[ENGINE] | LINK | POLICE | BOMB | NOTE
SLIP | [GOLD] | [SCALE] | [CAPITAL] | [POOL]
[DATE] | [FILE] | WHALE | [NOVEL] | [WAVE]
[STAFF] | LIFE | CZECH | CRASH | [SUIT]


=== Turn 12 ===
Remaining words to guess: NOTE, NEW YORK


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


codemaster raw output: ```json
{
  "reasoning": "The clue should connect two team words, NOTE and NEW YORK, and avoid the Assassin word, LIFE. 'DATE' was a team word in the previous round, and 'CALENDAR' is a related concept.",
  "clue": "CALENDAR",
  "number": 2
}
Codemaster's clue: CALENDAR 2
(Reasoning: The clue should connect two team words, NOTE and NEW YORK, and avoid the Assassin word, LIFE. 'DATE' was a team word in the previous round, and 'CALENDAR' is a related concept.)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


guesser raw output: ```json
{
  "reasoning": "Based on the clue 'CALENDAR 2' from the previous turn, one of the words was 'DATE'. Considering the clue 'CALENDAR 2' in this turn, another word related to 'CALENDAR' is likely to be 'NEW YORK', as it can be associated with a calendar's year.",
  "guess": "NEW YORK"
}
Guesser's guess: NEW YORK
(Reasoning: Based on the clue 'CALENDAR 2' from the previous turn, one of the words was 'DATE'. Considering the clue 'CALENDAR 2' in this turn, another word related to 'CALENDAR' is likely to be 'NEW YORK', as it can be associated with a calendar's year.)
Correct guess!
Remaining guesses this turn: 2
guesser raw output: ```json
{
  "reasoning": "Given that the clue is 'CALENDAR 2' and the words 'DATE' and 'WAVE' were found, I'm looking for words related to the concept of a calendar. 'WAVE' could be related to a wave in the ocean, but it could also be a wave in a calendar, referring to a wave symbol used in some calendars. However, since 'WAVE' has a m